## 2a. Calculate closed-system degassing path assuming melt composition is the bulk composition (with and without sulfur saturation)

## Setting things up
First we need to import a few Python packages (including VolFe) and set the precision of the calculations. You need to install VolFe once on your machine, if you haven't yet, uncomment the line below (remove the #).

In [ ]:
# Install VolFe on your machine. Don't remove the # from this line!
# pip install VolFe # Remove the first # in this line if you have not installed VolFe on your machine before.

# import python packages
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import VolFe as vf

At a minimum to run a degassing calculations, we need a dataframe of the melt composition, an estimate of oxygen fugacity and Fe in the melt [note 1], and temperature at some point along the degassing path:

- Sample is just the name for this analysis.

- Temperature is in 'C.

- Volatile-free melt composition is in wt% oxides [note 2]. All these oxides must be present in the dataframe, so set them to 0. if you have no data for them (although see [note 1] around FeOT).

- H2O is all hydrogen in the melt reported as H2O in wt% [note 3]. 

- CO2ppm is all carbon in the melt reported as CO2 in ppm [note 3].

- STppm is all sulfur in the melt reported as S in ppm [note 3].

- Xppm is all "X" in the melt reported as X in ppm [note 3]. X is an unreactive melt species whose identity can be changed - this is explored WHERE.

- Fe3+FeT is the ratio of Fe3+ to FeT in the melt.

[note 1] In this example we specify oxygen fugacity using Fe3+/FeT and Fe in the melt as FeOT as is quite common for melt inclusion and matrix glass analyses - other options are possible, see Example 1a for more information.

[note 2] It does not matter what the non-volatile oxides sum too - they are renormalised to 100 wt% minus the total of the volatiles (i.e., H2O + CO2 + ST + X).

[note 3] The volatile concentrations are absolute for the melt = i.e., the non-volatile melt composition is normalised to 100 wt% minus the volatiles. The volatiles are not added to the oxides and then renormalised.

The following composition is analysis Sari15-04-33 from Brounce et al. (2014), with a temperature chosen as 1200 'C.

In [ ]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Sari15-04-33',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 47.89, # wt%
           'TiO2': 0.75, # wt%
           'Al2O3': 16.74, # wt%
           'FeOT': 9.43, # wt%
           'MnO': 0.18, # wt%
           'MgO': 5.92, # wt%
           'CaO': 11.58, # wt%
           'Na2O': 2.14, # wt%
           'K2O': 0.63, # wt%
           'P2O5': 0.17, # wt%
           'H2O': 4.17, # wt%
           'CO2ppm': 1487., # ppm
           'STppm': 1343.5, # ppm
           'Xppm': 0., # ppm
           #'Fe3FeT': 0.195,
           'S6ST':0.5}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

For this example we will mostly use the default options in VolFe, which can be found below:

In [ ]:
# print default options in VolFe
print(vf.default_models)

But I'll just highlight the specific options that are important to the degassing calculation and what their default values are:

**bulk_composition = yes** This means the composition (including volatiles and fO2 estimate) represent the bulk composition of the system - there is only melt present and all volatiles are dissolved in the melt.

**starting_P = bulk** This means the calculation will start at Pvsat for the composition given.

**gasssing_style = closed** This means the melt and vapor remain in chemical equilibrium throughout and the bulk composition is constant during the calculation

**gassing_direction = degas** This means pressure will decrease during the calculations and therefore the melt will degas.

The default pressure step is 10 bars, which works most of the time but can be a little slow.
Increasing the step size will speed up the calculation but if the step size is too big it will struggle to solve because it uses the previous step for its initial guesses.

In [ ]:
degas1 = vf.calc_gassing(my_analysis)

We could run the same calculation from a csv file if we preferred.
We'll use the examples_marianas csv in files and use all the default options again. 
The data in this file are from Brounce et al. (2014, 2016) and Kelley & Cottrell (2012).
Sari15-04-33 is row 48 in that file. 
Uncomment the lines to use them (i.e., remove the # at the start of each line).

In [ ]:
# my_analyses = pd.read_csv("../files/example_marianas.csv") 

# vf.calc_gassing(my_analyses,row=48)

Turns out the melt was sulfide-saturated during the degassing path. 
Under those conditions, the results for the melt and vapor are metastable with respect to saturation of sulfide.

We can run the same calculation, but instead limit the sulfur content in the melt to sulfide or anhydrite saturation, if it reaches the limit. 
To do that we change the "sulfur_saturation" option from "no" (sulfur phases cannot saturate) to "yes" (sulfur phases can saturate if the sulfur content is high enough).

In [ ]:
# Choose the options I want to change for the calculation - everything else will use the default options
my_models = [['sulfur_saturation','yes']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

And then we run the degassing calculation with those model options

In [ ]:
degas2 = vf.calc_gassing(my_analysis, models=my_models)

And below we can plot the two results to see the difference.
The blue curves are the first calculation (not allowing sulfur to saturate), whilst the red curves are the second calculation (sulfur can saturate as sulfide or anhydrite if the concentration in the melt is high enough).
It has no effect on the H2O or CO2 contents of the melt, but the sulfur content is much less at the start of degassing because it is in a sulfide phase instead!

In [ ]:
fig = make_subplots(rows=1, cols=3, shared_yaxes = True, shared_xaxes = False,vertical_spacing=0.08, horizontal_spacing=0.05)

lw = 3
syms = 10
lc1 = "steelblue"
lc2 = "firebrick"

data1 = degas1 # sulfur cannot saturate
data2 = degas2 # sulfur can saturate

r = 1
# P vs. CO2
c = 1
fig.add_trace(go.Scatter(mode = "lines", x=data1['CO2T-eq_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['CO2T-eq_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "CO<sub>2</sub>-eq (ppmw)", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", autorange = 'reversed', row = r, col = c)

# P vs. H2O
c = 2
fig.add_trace(go.Scatter(mode = "lines", x=data1['H2OT-eq_wtpc'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['H2OT-eq_wtpc'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "H<sub>2</sub>O-eq (wt%)", row = r, col = c)

# P vs. S
c = 3
fig.add_trace(go.Scatter(mode = "lines", x=data1['ST_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['ST_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "S<sub>T</sub> (ppmw)", row = r, col = c)

fig.update_layout(height=400, width=1000, plot_bgcolor='rgb(255,255,255)' , showlegend = False)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))

fig.update_layout(font_family="Helvetica",font_color="black")

fig.show()